In [8]:
import time
import os
import csv
from dotenv import load_dotenv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

load_dotenv()
FACEBOOK_EMAIL = os.getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = os.getenv('FACEBOOK_PASSWORD')
PAGE_URL = 'https://m.facebook.com/groups/322977734828852'
OUTPUT_CSV_FILE = 'Facebook_group_post_urls_last5y.csv'
PROFILE_PATH = r'C:\chrome-profiles\fb-pipeline-stage1-persistent'

def login_to_facebook(driver):
    driver.get("https://m.facebook.com")
    time.sleep(3)
    selectors = [
        "button[data-cookiebanner='accept_button_dialog']",
        "button[title='Allow all cookies']",
        "button[title='Accept All']",
        "button[aria-label='Allow all cookies']"
    ]
    for s in selectors:
        btns = driver.find_elements(By.CSS_SELECTOR, s)
        if btns and btns[0].is_displayed():
            btns[0].click()
            time.sleep(2)
            break
    email = driver.find_elements(By.ID, "m_login_email")
    pwd = driver.find_elements(By.ID, "m_login_password")
    if not email:
        email = driver.find_elements(By.ID, "email")
        pwd = driver.find_elements(By.ID, "pass")
    if email and pwd:
        email[0].clear()
        email[0].send_keys(FACEBOOK_EMAIL)
        pwd[0].send_keys(FACEBOOK_PASSWORD)
        pwd[0].send_keys(Keys.RETURN)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[aria-label='Menu'], a[aria-label='Home'], a[aria-label='Search']")))

def collect_group_posts_last_5y(driver, group_url):
    cutoff_epoch = int(time.time() - 5 * 365 * 24 * 60 * 60)
    driver.get(group_url)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='main'], #m_group_stories_container, article")))
    time.sleep(3)
    group_id = group_url.rstrip('/').split('/')[-1]
    urls = set()
    prev_len = 0
    stagnant = 0
    reached_old = False
    loops = 0
    while True:
        data = driver.execute_script("""
            var gid = arguments[0];
            var anchors = Array.from(document.querySelectorAll("a[href*='/groups/']"));
            var out = [];
            for (var i=0;i<anchors.length;i++){
                var a = anchors[i];
                var href = a.getAttribute("href") || "";
                if(!href) continue;
                if(href.indexOf("/groups/"+gid+"/") === -1 && href.indexOf("story.php") === -1) continue;
                if(href.indexOf("/reel") !== -1 || href.indexOf("/reels/") !== -1 || href.indexOf("/videos/") !== -1 || href.indexOf("/watch/") !== -1 || href.indexOf("video_id=") !== -1) continue;
                if(href.indexOf("/permalink/") === -1 && href.indexOf("/posts/") === -1 && href.indexOf("story.php") === -1) continue;
                if(href.indexOf("http")!==0){ href = location.origin + href; }
                href = href.split("?")[0];
                var el = a.closest("article") || a.closest("div");
                var ut = null;
                if(el){
                    var t = el.querySelector("abbr[data-utime], time[data-utime]");
                    if(t){ ut = parseInt(t.getAttribute("data-utime")); }
                    if(!ut){
                        var t2 = el.querySelector("time[datetime], abbr[datetime]");
                        if(t2 && t2.getAttribute("datetime")){
                            var ms = Date.parse(t2.getAttribute("datetime"));
                            if(!isNaN(ms)) ut = Math.floor(ms/1000);
                        }
                    }
                }
                out.push([href, ut]);
            }
            return out;
        """, group_id)
        for href, ut in data:
            if ut is None or ut >= cutoff_epoch:
                urls.add(href)
        if any((ut is not None and ut < cutoff_epoch) for _, ut in data):
            reached_old = True
        see_more = driver.find_elements(By.XPATH, "//div[@role='button' and (contains(., 'See more') or contains(., 'See More'))]")
        if see_more:
            see_more[0].click()
            time.sleep(1.2)
        driver.execute_script("window.scrollBy(0, arguments[0]);", 1200)
        time.sleep(2.0)
        curr_len = len(urls)
        if curr_len == prev_len:
            stagnant += 1
        else:
            stagnant = 0
        prev_len = curr_len
        loops += 1
        if (reached_old and stagnant >= 2) or stagnant >= 6 or loops >= 3000:
            break
    return urls

def main():
    options = uc.ChromeOptions()
    options.add_argument("--disable-notifications")
    options.add_argument("--lang=en-US")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument(f"--user-data-dir={PROFILE_PATH}")
    options.page_load_strategy = "eager"
    driver = uc.Chrome(options=options, use_subprocess=True)
    driver.set_page_load_timeout(90)
    driver.set_script_timeout(90)
    driver.command_executor._client_config.timeout = 300
    login_to_facebook(driver)
    all_urls = collect_group_posts_last_5y(driver, PAGE_URL)
    driver.quit()
    if all_urls:
        with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
            w = csv.writer(f)
            w.writerow(['PostURL'])
            for u in sorted(all_urls):
                w.writerow([u])

if __name__ == "__main__":
    main()

TimeoutException: Message: timeout: Timed out receiving message from renderer: 90.000
  (Session info: chrome=140.0.7339.129)
Stacktrace:
	GetHandleVerifier [0x0xa7c333+65459]
	GetHandleVerifier [0x0xa7c374+65524]
	(No symbol) [0x0x89d973]
	(No symbol) [0x0x88e184]
	(No symbol) [0x0x88deba]
	(No symbol) [0x0x88bdc0]
	(No symbol) [0x0x88c8a7]
	(No symbol) [0x0x8994fe]
	(No symbol) [0x0x8aaec5]
	(No symbol) [0x0x8b0a56]
	(No symbol) [0x0x88cf26]
	(No symbol) [0x0x8aab6e]
	(No symbol) [0x0x8aa921]
	(No symbol) [0x0x92b515]
	(No symbol) [0x0x909bf6]
	(No symbol) [0x0x8db38e]
	(No symbol) [0x0x8dc274]
	GetHandleVerifier [0x0xcfeda3+2697763]
	GetHandleVerifier [0x0xcf9ec7+2677575]
	GetHandleVerifier [0x0xaa4194+228884]
	GetHandleVerifier [0x0xa949f8+165496]
	GetHandleVerifier [0x0xa9b18d+192013]
	GetHandleVerifier [0x0xa847d8+99416]
	GetHandleVerifier [0x0xa84972+99826]
	GetHandleVerifier [0x0xa6ebea+10346]
	BaseThreadInitThunk [0x0x765d5d49+25]
	RtlInitializeExceptionChain [0x0x772cd6db+107

In [2]:
import time
import os
import csv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException

FACEBOOK_EMAIL = os.getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = os.getenv('FACEBOOK_PASSWORD')

PAGE_URL = 'https://www.facebook.com/anantajewelry'
OUTPUT_CSV_FILE = 'anantajewelry.csv'
PROFILE_PATH = r'C:\chrome-profiles\fb-scraper-profile'

def login_to_facebook(driver):
    driver.get("https://www.facebook.com")
    time.sleep(3)
    
    cookie_buttons = driver.find_elements(By.CSS_SELECTOR, "button[data-cookiebanner='accept_button_dialog']")
    if cookie_buttons and cookie_buttons[0].is_displayed():
        cookie_buttons[0].click()
        time.sleep(2)
        
    email_input_list = driver.find_elements(By.ID, "email")
    if email_input_list:
        pass_input = driver.find_element(By.ID, "pass")
        email_input_list[0].send_keys(FACEBOOK_EMAIL)
        pass_input.send_keys(FACEBOOK_PASSWORD)
        pass_input.submit()
        time.sleep(5)

def scrape_page_feed(driver, page_url):
    driver.get(page_url)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='main']")))
    time.sleep(3)

    print("Scrolling the page feed to load posts...")
    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

    all_scraped_profiles = {}
    reactors_button_selector = "div.x78zum5.xdt5ytf span.xt0b8zv.x1jx94hy"
    
    post_reaction_buttons = driver.find_elements(By.CSS_SELECTOR, reactors_button_selector)
    print(f"Found {len(post_reaction_buttons)} posts with reaction buttons to process.")

    for i in range(len(post_reaction_buttons)):
        buttons = driver.find_elements(By.CSS_SELECTOR, reactors_button_selector)
        if i >= len(buttons):
            break
        button = buttons[i]
        
        try:
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", button)
            print(f"\nProcessing post {i + 1}...")

            dialog_selector = "div[role='dialog']"
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, dialog_selector)))

            no_change_count = 0
            while no_change_count < 3:
                profiles_before_scrape = len(all_scraped_profiles)
                
                profile_elements = driver.find_elements(By.CSS_SELECTOR, "div[role='dialog'] span.xjp7ctv > a")

                for element in profile_elements:
                    name = element.text
                    url = element.get_attribute('href')
                    if name and url:
                        clean_url = url.split('?')[0]
                        all_scraped_profiles[clean_url] = {'profile_name': name, 'profile_url': clean_url}
                
                dialog = driver.find_element(By.CSS_SELECTOR, dialog_selector)
                driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', dialog)
                time.sleep(2.5)
                
                profiles_after_scrape = len(all_scraped_profiles)
                if profiles_after_scrape == profiles_before_scrape:
                    no_change_count += 1
                else:
                    no_change_count = 0
            
            print(f"Scraped from post {i + 1}. Total unique profiles so far: {len(all_scraped_profiles)}")
            
            ActionChains(driver).send_keys(Keys.ESCAPE).perform()
            time.sleep(2)
        except TimeoutException:
            print(f"Skipping post {i + 1} as no reaction dialog appeared.")
            ActionChains(driver).send_keys(Keys.ESCAPE).perform()
            time.sleep(1)
            continue

    return list(all_scraped_profiles.values())

def main():
    os.makedirs(PROFILE_PATH, exist_ok=True)
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-notifications")
    options.add_argument("--lang=en-US")
    options.add_experimental_option('prefs', {'intl.accept_languages': 'en-US,en'})
    options.add_argument(f"--user-data-dir={PROFILE_PATH}")

    with uc.Chrome(options=options, use_subprocess=True) as driver:
        login_to_facebook(driver)
        reactors_data = scrape_page_feed(driver, PAGE_URL)

    if reactors_data:
        with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['profile_name', 'profile_url'])
            writer.writeheader()
            writer.writerows(reactors_data)
        print(f"\nScraping complete. Saved {len(reactors_data)} unique profiles to {OUTPUT_CSV_FILE}.")

if __name__ == "__main__":
    main()

Scrolling the page feed to load posts...
Found 1 posts with reaction buttons to process.

Processing post 1...
Scraped from post 1. Total unique profiles so far: 10

Scraping complete. Saved 10 unique profiles to anantajewelry.csv.


In [3]:
import time
import os
import csv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

FACEBOOK_EMAIL = os.getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = os.getenv('FACEBOOK_PASSWORD')
INPUT_CSV_FILE = 'anantajewelry.csv'
OUTPUT_CSV_FILE_WITH_CONTACTS = 'anantajewelryname.csv'
PROFILE_PATH = r'C:\chrome-profiles\fb-contact-scraper-final'
WEBDRIVER_WAIT_TIMEOUT = 15

if not FACEBOOK_EMAIL or not FACEBOOK_PASSWORD:
    print("Error: Set FACEBOOK_EMAIL and FACEBOOK_PASSWORD environment variables.")
    exit()

options = uc.ChromeOptions()
options.add_argument(f'--user-data-dir={PROFILE_PATH}')
options.add_argument('--no-first-run')
options.add_argument('--no-service-autorun')
options.add_argument('--password-store=basic')
options.add_argument('--disable-notifications')

driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)

driver.get("https://www.facebook.com")
time.sleep(3)

cookie_buttons = driver.find_elements(By.CSS_SELECTOR, 'button[data-cookiebanner="accept_button"]')
if cookie_buttons:
    cookie_buttons[0].click()
    time.sleep(2)

password_fields = driver.find_elements(By.ID, "pass")
if password_fields:
    email_input = wait.until(EC.presence_of_element_located((By.ID, "email")))
    pass_input = driver.find_element(By.ID, "pass")
    email_input.send_keys(FACEBOOK_EMAIL)
    pass_input.send_keys(FACEBOOK_PASSWORD)
    driver.find_element(By.NAME, "login").click()

wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="main"]')))

with open(OUTPUT_CSV_FILE_WITH_CONTACTS, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['Profile Name', 'Profile URL', 'Contact Info'])

    with open(INPUT_CSV_FILE, 'r', newline='', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        next(reader)

        for row in reader:
            profile_name, profile_url = row
            contact_page_url = ""
            
            if "profile.php" in profile_url:
                contact_page_url = f"{profile_url}&sk=about_contact_and_basic_info"
            else:
                base_url = profile_url.split('?')[0]
                contact_page_url = f"{base_url.rstrip('/')}/about_contact_and_basic_info"
            
            driver.get(contact_page_url)
            time.sleep(5)

            contact_info_text = "Not Found"
            contact_elements = driver.find_elements(By.CSS_SELECTOR, "div.xyamay9.xsfy40s.x1gan7if.xf7dkkf")
            
            if contact_elements:
                contact_info_text = contact_elements[0].text.replace('\n', ' | ')

            writer.writerow([profile_name, profile_url, contact_info_text])
            print(f"Scraped: {profile_name}")

driver.quit()
print(f"Scraping complete. Data saved to {OUTPUT_CSV_FILE_WITH_CONTACTS}")

Scraped: Jane Rungthiwa
Scraped: Iam Panna
Scraped: Takky Tak Pongudtha
Scraped: Bee BeeGun
Scraped: Maysa Bhakin
Scraped: Piyanuch Hombanyen
Scraped: Chonthicha Sukyot
Scraped: Kasidid Suksrikarn
Scraped: Dhita Parasari
Scraped: Mickey Mickey
Scraping complete. Data saved to anantajewelryname.csv
